In [12]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [32]:
def get_sizes_stats():
    annotations_dir = "/home/lowik/sealion/data/sealion/TrainAnnotations/"
    dots_dir = "/home/lowik/sealion/data/sealion/TrainDots/"
    sizes = [[] for _ in range(5)]
    
    for filename in os.listdir(annotations_dir):
        # Load annotation mask and dots
        train_id, _ = os.path.splitext(filename)
        train_id = int(train_id)
        mask = cv2.imread(os.path.join(annotations_dir, filename), cv2.IMREAD_GRAYSCALE)
        n_labels, im_labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
        with open(os.path.join(dots_dir, "{}.pkl".format(train_id)), "rb") as p:
            dots = pickle.load(p)
        
        # Find the sealion type for each label.
        blob_types = dict()
        for i, ds in enumerate(dots):
            for x, y in ds:
                if mask[y, x] != 0:
                    blob_types[im_labels[y, x]] = i
                    
        # Compute size of each blobs and add it to the corresponding sealion data.
        for label in range(1, n_labels):
            y, x = np.nonzero(im_labels == label)
            coords = np.zeros((x.shape[0], 2), dtype=np.int32)
            coords[:, 0] = x
            coords[:, 1] = y
            ((xc, yc), (w, h), a) = cv2.minAreaRect(coords)
            if label not in blob_types:
                print("{train_id} - label {label} has no type?".format(train_id=train_id, label=label))
                continue
            sizes[blob_types[label]].append(max(w, h))
            
    return sizes

In [63]:
sizes = get_sizes_stats()

In [64]:
for i, ss in enumerate(sizes):
    mi = np.min(ss)
    ma = np.max(ss)
    mean = np.mean(ss)
    std = np.std(ss)
    print("{}: min={}, max={}, mean={}, std={}, n={}".format(i, mi, ma, mean, std, len(ss)))

0: min=41.87986755371094, max=164.6144561767578, mean=81.30168822004988, std=22.956427884506798, n=37
1: min=45.04875564575195, max=142.55235290527344, mean=75.6908574785505, std=24.858394791602496, n=21
2: min=39.369667053222656, max=138.251953125, mean=62.66514139175415, std=19.768246623079616, n=40
3: min=35.0, max=102.97856140136719, mean=51.28458023071289, std=14.259899847392974, n=33
4: min=18.026947021484375, max=55.61539077758789, mean=29.880167219373917, std=8.777418843640895, n=18
